In [ ]:
# Testing Cell
import aviary.api as av
from aviary.interface.methods_for_level2 import AviaryGroup, AviaryProblem
from aviary.utils.doctape import glue_class_functions, glue_variable, glue_class_options

current_glued_vars = []

glue_variable(av.EquationsOfMotion.HEIGHT_ENERGY.value, md_code=False)
glue_variable(av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, md_code=False)

glue_class_options(AviaryGroup, current_glued_vars, md_code=True)

folder = 'examples/reserve_missions/'

str_level2_example = 'run_reserve_mission_multiphase.py'
file_path = av.get_path(folder + str_level2_example)
glue_variable(file_path.name, md_code=True)

# Get all functions of class AviaryProblem
glue_class_functions(AviaryProblem, current_glued_vars, prefix='prob')

# Reserve Mission

## Overview

Reserve missions are enabled for the following mission types:

* {glue:md}`height_energy`
* {glue:md}`2DOF`

A reserve mission can be created by appending one or more reserve phases to {glue:md}`mission_info` after the last phase of the regular mission.
To create a simple reserve mission, copy your current cruise phase which is located in {glue:md}`mission_info`.

```{note}
You may need to revise some of your assumptions for the copied phase if you are making a reserve phase that is radically different than the original (i.e. original phase was to travel 3000km but reserve phase is 100km).
```

Append that phase to the end of {glue:md}`mission_info`, name it `reserve_cruise` and add `"reserve": True,` to `user_options` for this phase.
There are two optional flags that can now be added to `user_options`.
The `"target_duration"` option creates a phase requiring the aircraft to fly for a specific amount of time.
The `"target_distance"` option creates a phase requiring the aircraft to fly for a specific distance.
Avoid using the optional flag if you have a reserve phase (i.e climb or descent) where you just want that phase to be completed as quickly as possible.
The optional flags should not be combined as they will create overlapping constraints creating an infeasible problem.

In [ ]:
# Testing Cell
import os
from importlib.machinery import SourceFileLoader

import aviary.api as av
from aviary.mission.flight_phase_builder import FlightPhaseBase
from aviary.utils.doctape import check_contains

gasp_phase_path = av.get_path(os.path.join('mission', 'two_dof', 'phases'))
files = os.listdir(gasp_phase_path)
phases = [FlightPhaseBase]
for file in files:
    if '_phase.py' in file and 'twodof' not in file:
        file_path = os.path.join(str(gasp_phase_path), file)
        phase_name = file.split('_phase.py')[0].capitalize()
        module = SourceFileLoader(phase_name, file_path).load_module()
        phases.append(getattr(module, phase_name + 'Phase'))

for phase in phases:
    phase_name = phase().__class__.__name__
    if 'Groundroll' in phase_name:
        continue  # no reserve groundroll
    check_contains(
        ['reserve', 'time_duration', 'target_distance'],
        phase.default_options_class(),
        error_string='{var} is not a valid key for ' + str(phase_name),
    )

You can chain together multiple reserve phases to make a complete reserve mission (i.e. climb to altitude, cruise for range, cruise for time, then descend).
An example of this is shown in {glue:md}`run_reserve_mission_multiphase.py`.

In [ ]:
# Testing Cell
import os

import aviary.api as av
from aviary.utils.doctape import check_contains

reserve_examples = av.get_path(os.path.join('examples', 'reserve_missions'))
check_contains(
    ('run_reserve_mission_multiphase.py'),
    os.listdir(reserve_examples),
    error_string='{var} not in ' + str(reserve_examples),
    error_type=FileNotFoundError,
)

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import glue_variable

methods_for_level2 = av.get_path('interface/methods_for_level2.py').name
glue_variable(methods_for_level2, md_code=True)

setting_2DOF = (
    av.Settings.EQUATIONS_OF_MOTION + ',' + av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value
)
glue_variable('setting_2DOF', setting_2DOF, md_code=True)

The first reserve phase will start at the same range and mass as the last regular phase, but all other states (i.e. altitude, Mach number) are not automatically connected.
Thus you can fly climb, cruise, descent for regular phases and then immediately jump to an arbitrary altitude for the reserve mission.
Or if you wanted to make things more realistic you could attach a climb phase and then add your reserve cruise.
Make sure both the reserve climb and the reserve cruise phases both have `"reserve": True,` flag.

### Examples

Examples of single-phase and multi-phase reserve missions are presented in [Reserve Mission Examples](../examples/reserve_missions).

### Caveats when using 2DOF

If you are using {glue:md}`2DOF` equations of motion (EOM) in your problem (i.e. {glue:md}`setting_2DOF`) there are some additional things you need to be aware of.
The name of the reserve phase should include one of the keywords to indicate which EOM from {glue:md}`2DOF` will be selected and the prefix `reserve_`.
Valid keywords include: {glue:md}`rotation`, {glue:md}`accel`, {glue:md}`ascent`, {glue:md}`climb1`, {glue:md}`climb2`, {glue:md}`cruise`, {glue:md}`desc1`, {glue:md}`desc2`.
This is because {glue:md}`2DOF` uses different EOMs for different phases and we need to let {glue:md}`methods_for_level2.py` know which method to select.
This is why in the example in the first paragraph above, the phase was named `reserve_cruise`.
Cruise phases can have additional information in suffixes, but this isn't necessary.
Do not worry about phase naming if you are using Height-Energy EOM as all those EOMs are the same for every phase.

In [ ]:
# Testing Cell
from aviary.utils.doctape import check_contains, glue_variable

two_DOF_phases = ['rotation', 'accel', 'ascent', 'climb1', 'climb2', 'cruise', 'desc1', 'desc2']
for ph in two_DOF_phases:
    glue_variable(ph, md_code=True)

## Theory

When adding a reserve phase, {glue:md}`check_and_preprocess_inputs()` divides all the phases into two dictionaries: `regular_phases` which contain your nominal phases and `reserve_phases` which contains any phases with the `reserve` flag set to `True`.
Additionally, {glue:md}`check_and_preprocess_inputs()` will add the `"analytic"` flag to each phase.
This is used to indicate if a phase is an analytic phase (i.e. Breguet range) or a ordinary differential equation (ODE).

Only the final mission mass and range from `regular_phases` are automatically connected to the first point of the `reserve_phases`.
All other state variables (i.e. altitude, mach) are not automatically connected, allowing you to start the reserve mission at whatever altitude you want.

The `"analytic"` flag helps to properly connect phases for {glue:md}`2DOF` missions.
{glue:md}`2DOF` `cruise` missions are analytic because they use a Breguet range calculation instead of integrating an EOM. 
Analytic phases have a slightly different naming convention in order to access state/timeseries variables like distance, mass, and range compared with their non-analytic counterparts.

You cannot create a reserve mission that enforces time or range constraints over multiple phases (i.e specify the total range covered by a climb + cruise + descent).
This is because each constraint `"target_distance"` or `"target_time"` is only enforced on a single phase.
It is essential that you run {glue:md}`prob.check_and_preprocess_inputs()` after {glue:md}`prob.load_inputs()` to make sure that regular and reserve phases are separated via `phase_separator()`.

In [ ]:
# Testing Cell
import os
from importlib.machinery import SourceFileLoader

import aviary.api as av
from aviary.models.missions.two_dof_default import phase_info
from aviary.interface.download_models import get_model
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.utils.doctape import check_contains

prob = AviaryProblem()
prob.load_inputs(get_model('aircraft_for_bench_GwGm.csv'), phase_info)
prob.check_and_preprocess_inputs()

gasp_phase_path = av.get_path(os.path.join('mission', 'two_dof', 'phases'))
for file in os.listdir(gasp_phase_path):
    if '_phase.py' in file and 'twodof' not in file:
        phase_name = file.split('_phase.py')[0].capitalize()
        file_path = os.path.join(str(gasp_phase_path), file)
        module = SourceFileLoader(phase_name, file_path).load_module()
        check_contains(
            'analytic',
            getattr(module, phase_name + 'PhaseOptions')(),
            error_string=f'analytic is not a valid key for {phase_name}',
            error_type=NameError,
        )

### Advanced Users and Target Duration Phases

For advanced users, instead of just copying a phase you used before, you might completely specify a new phase from scratch. 
When creating a `"target_duration"` reserve phase there are a number of values inside of `phase_info['user_options']` that are overwritten in {glue:md}`prob.check_and_preprocess_inputs()`. 
Specifically, `duration_bounds`, `fixed_duration`, and `"initial_guesses": {"time"}` will be over-written. 
That is because if `"target_duration"` is specified, Aviary already knows what these other three values need to be: `target_duration = duration_bounds = "initial_guesses": {"time"}`, and `fix_duration = True`.

In [ ]:
# Testing Cell
from copy import deepcopy

from aviary.models.missions.two_dof_default import phase_info
from aviary.interface.download_models import get_model
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.utils.doctape import check_value

climb1_info = deepcopy(phase_info['climb1'])
phase_info_for_test = {'climb1': climb1_info}
user_options = climb1_info['user_options']
user_options['reserve'] = True
user_options['time_duration'] = (10, 'min')
user_options['time_duration_bounds'] = ((30, 300), 's')
climb1_info['initial_guesses']['time'] = ((1.0, 2.0), 'min')

prob = AviaryProblem()
prob.load_inputs(get_model('aircraft_for_bench_GwGm.csv'), phase_info_for_test)

prob.check_and_preprocess_inputs()

values_of_interest = {
    'time_duration': user_options['time_duration'],
    'time_duration_bounds': user_options['time_duration_bounds'],
    'time': climb1_info['initial_guesses']['time'],
}
expected_values = {
    'time_duration': (10, 'min'),
    'time_duration_bounds': ((30, 300), 's'),
    'time': ((1.0, 10.0), 'min'),
}
check_value(values_of_interest, expected_values)

### Fuel Burn Calculations

Fuel burn during the regular mission ({glue:md}`Mission.Summary.FUEL_BURNED`) is calculated only based on `regular_phases`.

Reserve fuel ({glue:md}`Mission.Design.RESERVE_FUEL`) is the sum of {glue:md}`Aircraft.Design.RESERVE_FUEL_ADDITIONAL`, {glue:md}`Aircraft.Design.RESERVE_FUEL_FRACTION`, and {glue:md}`Mission.Summary.RESERVE_FUEL_BURNED`.

* {glue:md}`RESERVE_FUEL_ADDITIONAL` is a fixed value (i.e. 300kg)
* {glue:md}`RESERVE_FUEL_FRACTION` is based on a fraction of {glue:md}`Mission.Summary.FUEL_BURNED`
* {glue:md}`RESERVE_FUEL_BURNED` is sum of fuel burn in all `reserve_phases`


In [ ]:
# Testing Cell
from aviary.api import Aircraft, Mission
from aviary.utils.doctape import get_variable_name, glue_variable

Mission.Summary.FUEL_BURNED
Mission.Design.RESERVE_FUEL
Aircraft.Design.RESERVE_FUEL_ADDITIONAL
Aircraft.Design.RESERVE_FUEL_FRACTION
glue_variable(get_variable_name(Mission.Summary.FUEL_BURNED), md_code=True)
glue_variable(get_variable_name(Mission.Design.RESERVE_FUEL), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.RESERVE_FUEL_ADDITIONAL), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.RESERVE_FUEL_FRACTION), md_code=True)
glue_variable(get_variable_name(Mission.Summary.RESERVE_FUEL_BURNED), md_code=True)
glue_variable(
    get_variable_name(Aircraft.Design.RESERVE_FUEL_ADDITIONAL).split('.')[2], md_code=True
)
glue_variable(get_variable_name(Aircraft.Design.RESERVE_FUEL_FRACTION).split('.')[2], md_code=True)
glue_variable(get_variable_name(Mission.Summary.RESERVE_FUEL_BURNED).split('.')[2], md_code=True)